In [1]:
import os
import shutil
import pandas as pd
import numpy as np

In [4]:
lidc = pd.read_csv('data/LIDC_20130817_AllFeatures2D_MaxSlicePerNodule_inLineRatings.csv')

In [5]:
lidc.head(5)

,InstanceID,StudyInstanceUID,SeriesInstanceUid,RadiologistID,noduleID,DicomImage,FilePath,subtlety,internalStructure,calcification,...,Correlation,Energy,Homogeneity,Entropy,x_3rdordermoment,Inversevariance,Sumaverage,Variance,Clustertendency,MaxProbability
0,4,1.3.6.1.4.1.14519.5.2.1.6279.6001.298806137288...,1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636...,1,1,present,C:\LIDC_FULL\LIDC-IDRI\LIDC-IDRI-0001\1.3.6.1....,5,1,6,...,0.894,0.0003,0.0532,8.15,1290000,0.0223,661.0,143000,541000,0.0009
1,75,1.3.6.1.4.1.14519.5.2.1.6279.6001.490157381160...,1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417...,4,2,present,C:\LIDC_FULL\LIDC-IDRI\LIDC-IDRI-0002\1.3.6.1....,1,1,6,...,0.781,0.0005,0.0567,7.76,11100000,0.0221,260.0,32700,115000,0.0012
2,134,1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,4,3,present,C:\LIDC_FULL\LIDC-IDRI\LIDC-IDRI-0003\1.3.6.1....,5,1,6,...,0.899,0.0004,0.0572,8.03,735000,0.0247,416.0,111000,420000,0.0010
3,143,1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,4,4,present,C:\LIDC_FULL\LIDC-IDRI\LIDC-IDRI-0003\1.3.6.1....,3,1,6,...,0.692,0.0027,0.0312,6.03,-469000,0.0107,420.0,98700,333000,0.0043
4,118,1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276...,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,3,5,present,C:\LIDC_FULL\LIDC-IDRI\LIDC-IDRI-0003\1.3.6.1....,4,1,6,...,0.793,0.0018,0.0764,6.43,8600000,0.0392,565.0,137000,492000,0.0052


In [6]:
lidc_malig = lidc[['Malignancy_1','Malignancy_2','Malignancy_3','Malignancy_4']]

In [9]:
lidc['malignancy'] =lidc_malig.mask(lidc_malig == 0).mode(axis=1).max(axis=1)

In [18]:
lidc_spic = lidc[['Spiculation_1','Spiculation_2','Spiculation_3','Spiculation_4']]

In [21]:
lidc['spiculation'] = lidc_spic.mask(lidc_spic == 0).mode(axis=1).mean(axis=1).apply(np.ceil)


In [8]:
image_folder = 'LIDC(MaxSlices)_Nodules'
new_image_folder = 'LIDC(MaxSlices)_Nodules(fixed)'

noduleID = []

for file in os.scandir(image_folder): #might need to run multiple times

    #find the nodule malignancy using unique ID
    temp_nodule_ID = file.name.split('.')[0]
    malignancy = int(lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0])
    
    if malignancy != 3:
        noduleID.append(int(temp_nodule_ID))
        
    #comment out if don't want to move them
    #move image to correct subdirectory
    subdir = f'Malignancy_{malignancy}'
    shutil.copy(file, os.path.join(new_image_folder, subdir, os.path.basename(os.path.normpath(file))))


In [9]:
len(noduleID)

1515

In [24]:
nodules = lidc.loc[lidc['noduleID'].isin(noduleID)][['noduleID', 'malignancy', 'spiculation']]

In [25]:
nodules.head(5)

,noduleID,malignancy,spiculation
0,1,5.0,5.0
1,2,5.0,1.0
2,3,5.0,2.0
3,4,4.0,2.0
4,5,5.0,1.0


In [27]:
malignancy_b = ['malignant' if node > 3 else 'benign' for node in nodules['malignancy']]
spiculation_b = ['unmarked' if node < 2 else 'marked' for node in nodules['spiculation']]

In [28]:
nodules['malignancy_b'] = malignancy_b
nodules['spiculation_b'] = spiculation_b

In [29]:
nodules.head(5)

,noduleID,malignancy,spiculation,malignancy_b,spiculation_b
0,1,5.0,5.0,malignant,marked
1,2,5.0,1.0,malignant,unmarked
2,3,5.0,2.0,malignant,marked
3,4,4.0,2.0,malignant,marked
4,5,5.0,1.0,malignant,unmarked


In [30]:
nodules.groupby(['spiculation_b', 'malignancy_b']).size()

spiculation_b  malignancy_b
marked         benign          101
               malignant       407
unmarked       benign          725
               malignant       282
dtype: int64

In [31]:
725 / (725+101)

0.8777239709443099

In [32]:
407 / (407+282)

0.590711175616836

In [33]:
(725+407) / (725+101+407+282)

0.7471947194719472

In [37]:
nodules['subgroup'] = [f'{spic}_{malig}' for spic,malig in zip(nodules['spiculation_b'], nodules['malignancy_b'])]

In [38]:
nodules.head(5)

,noduleID,malignancy,spiculation,malignancy_b,spiculation_b,subgroup
0,1,5.0,5.0,malignant,marked,marked_malignant
1,2,5.0,1.0,malignant,unmarked,unmarked_malignant
2,3,5.0,2.0,malignant,marked,marked_malignant
3,4,4.0,2.0,malignant,marked,marked_malignant
4,5,5.0,1.0,malignant,unmarked,unmarked_malignant


In [39]:
nodules.to_csv('lidc_spic_subgrouped.csv')